# **Fase 03 — prepareWindowsDS**

### Generación eficiente de ventanas OW–LT–PW para variantes del pipeline MLOps4OFP

Este notebook:
- Carga parámetros declarativos de la variante actual (solo OW, LT, PW, estrategia, nan_strategy…)
- Localiza y carga automáticamente el dataset de eventos producido en Fase 02
- Resuelve `Tu` siguiendo la misma regla que las fases anteriores
  - `Tu` explícito en params.yaml → prioridad máxima
  - si no → `Tu` heredado del padre (Fase 02)
  - si no → `Tu` global (params.yaml raíz)
- Construye ventanas mediante algoritmos optimizados basados en índices y búsqueda binaria
- Genera dataset de ventanas (parquet), metadatos (json) y un informe HTML
- Registra git, timestamp y contexto completo de trazabilidad

⚡ Está diseñado para datasets MUY GRANDES: evita loops Python en lo posible.

In [1]:
# =====================================================================
# 1. IMPORTS COMUNES
# =====================================================================
import os
import sys
import json
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timezone
import seaborn as sns
import shutil

# Estilo gráfico común
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

print("✔ Imports y estilo cargados")

✔ Imports y estilo cargados


In [2]:
# =====================================================================
# 2. CONTEXTO DE EJECUCIÓN (bootstrap + run_context)
# =====================================================================
import sys
from pathlib import Path

# --- BOOTSTRAP MÍNIMO (antes de cualquier import mlops4ofp) ---
execution_dir = Path.cwd().resolve()

current = execution_dir
for _ in range(10):
    if (current / "mlops4ofp").exists():
        project_root = current
        break
    current = current.parent
else:
    raise RuntimeError("❌ No se pudo localizar project_root")

sys.path.insert(0, str(project_root))

print(f"📁 Project root añadido a PYTHONPATH: {project_root}")

# --- AHORA SÍ: imports normales ---
from mlops4ofp.tools.run_context import (
    detect_execution_dir,
    detect_project_root,
    assemble_run_context,
)
import yaml

PHASE = "03_preparewindowsds"

# -------------------------------------------------------------------
# Selección de variante
# -------------------------------------------------------------------
ACTIVE_VARIANT = os.environ.get("ACTIVE_VARIANT")

# Caso 1: ejecución vía Makefile (preferido)
if ACTIVE_VARIANT:
    print(f"[INFO] Variante activa (desde entorno): {ACTIVE_VARIANT}")

# Caso 2: notebook abierto manualmente
else:
    print("⚠️ ACTIVE_VARIANT no definido — seleccionando variante automáticamente…")

    variants_file = project_root / "params" / PHASE / "variants.yaml"

    if not variants_file.exists():
        raise RuntimeError(
            "❌ No existen variantes todavía.\n"
            "Crea una con:\n"
            "    make variant1 VARIANT=v001 RAW=tu_dataset.csv"
        )

    with open(variants_file, "r", encoding="utf-8") as f:
        reg = yaml.safe_load(f) or {}

    all_variants = sorted(reg.get("variants", {}).keys())
    if not all_variants:
        raise RuntimeError("❌ No hay variantes definidas en variants.yaml")

    ACTIVE_VARIANT = all_variants[-1]
    print(f"[INFO] Usando variante más reciente: {ACTIVE_VARIANT}")

# -------------------------------------------------------------------
# Construcción del contexto final
# -------------------------------------------------------------------
from mlops4ofp.tools.params_manager import ParamsManager

pm = ParamsManager(PHASE, project_root)

print("PHASE_DIR según ParamsManager:", pm.phase_dir)
print("¿Existe PHASE_DIR?", pm.phase_dir.exists())
print("¿Existe variante?", (pm.phase_dir / ACTIVE_VARIANT).exists())

pm.set_current(ACTIVE_VARIANT)

variant_root = pm.current_variant_dir()

ctx = assemble_run_context(
    execution_dir=detect_execution_dir(),
    project_root=project_root,
    phase=PHASE,
    variant=ACTIVE_VARIANT,
    variant_root=variant_root,
)

print("✔ Contexto de ejecución construido")
print(f"   Fase: {ctx['phase']}")
print(f"   Variante: {ctx['variant']}")
print(f"   Carpeta variante: {ctx['variant_root']}")


📁 Project root añadido a PYTHONPATH: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp
[INFO] Variante activa (desde entorno): v111
PHASE_DIR según ParamsManager: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds
¿Existe PHASE_DIR? True
¿Existe variante? True
✔ Contexto de ejecución construido
   Fase: 03_preparewindowsds
   Variante: v111
   Carpeta variante: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111


In [3]:
# =====================================================================
# 4. RUTAS DE DATOS Y SALIDAS (VARIANTE)
#    Todos los ficheros llevan como prefijo el nombre de la fase
# =====================================================================
from mlops4ofp.tools.run_context import build_phase_outputs
VARIANT_DIR = ctx["variant_root"]
PHASE_PREFIX = ctx["phase"]

# Datos de entrada (raw es compartido por todo el proyecto)
#RAW_DIR = ctx["project_root"] / "data" / "01-raw"
#RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUTS = build_phase_outputs(
    variant_root=VARIANT_DIR,
    phase=ctx["phase"],
)

# Figuras (única subcarpeta permitida dentro de la variante)
FIGURES_DIR = ctx["figures_dir"]

print("✔ Rutas de salida preparadas (scoped a la variante)")
OUTPUTS, FIGURES_DIR

✔ Rutas de salida preparadas (scoped a la variante)


({'dataset': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_dataset.parquet'),
  'report': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_report.html'),
  'params': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_params.json'),
  'metadata': PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_metadata.json')},
 PosixPath('/Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/figures'))

In [4]:
# =====================================================================
# 3. Leer parámetros de la variante activa (Fase 03)
# =====================================================================
import json
import yaml
from mlops4ofp.tools.params_manager import validate_params

params_path = VARIANT_DIR / "params.yaml"
if not params_path.exists():
    raise FileNotFoundError(
        f"No existe el fichero de parámetros de la variante:\n"
        f"  {params_path}\n"
        f"Primero crea la variante con:\n"
        f"  make variant3 VARIANT={ACTIVE_VARIANT} ..."
    )

# Leer parámetros + validar contra traceability_schema.yaml
with open(params_path, "r", encoding="utf-8") as f:
    params_f03 = yaml.safe_load(f) or {}

validate_params(PHASE, params_f03, ctx["project_root"])
print("✔ Parámetros de Fase 03 validados correctamente según schema.")

# ---------------------------------------------------------------------
# Parámetros propios de Fase 03
# ---------------------------------------------------------------------
OW = int(params_f03["OW"])
LT = int(params_f03["LT"])
PW = int(params_f03["PW"])

window_strategy = params_f03.get("window_strategy", "synchro")
nan_strategy = params_f03.get("nan_strategy", "discard")

parent_phase = params_f03.get("parent_phase", "02_prepareeventsds")
parent_variant = params_f03.get("parent_variant")
variant_id = params_f03.get("variant_id", ACTIVE_VARIANT)

# Validaciones mínimas explícitas (fase-específicas)
ALLOWED_WINDOW_STRATEGIES = {"synchro"}
if window_strategy not in ALLOWED_WINDOW_STRATEGIES:
    raise ValueError(
        f"window_strategy='{window_strategy}' no es válida.\n"
        f"Opciones permitidas: {ALLOWED_WINDOW_STRATEGIES}"
    )

ALLOWED_NAN_STRATEGIES = {"discard", "keep"}
if nan_strategy not in ALLOWED_NAN_STRATEGIES:
    raise ValueError(
        f"nan_strategy='{nan_strategy}' no es válida.\n"
        f"Opciones permitidas: {ALLOWED_NAN_STRATEGIES}"
    )

# Log compacto
print("────────── VARIANT PARAMETERS (F03) ──────────")
print(f"variant_id        = {variant_id}")
print(f"parent_phase      = {parent_phase}")
print(f"parent_variant    = {parent_variant}")
print(f"OW                = {OW}")
print(f"LT                = {LT}")
print(f"PW                = {PW}")
print(f"window_strategy   = {window_strategy}")
print(f"nan_strategy      = {nan_strategy}")
print("──────────────────────────────────────────────")

# =====================================================================
# 4. Cargar metadata de la Fase 02 (padre) para herencia de Tu
# =====================================================================
parent_metadata_path = (
    ctx["project_root"]
    / "params"
    / parent_phase
    / parent_variant
    / f"{parent_phase}_metadata.json"
)

if not parent_metadata_path.exists():
    raise FileNotFoundError(
        f"No se encontró metadata de la fase padre:\n{parent_metadata_path}"
    )

with open(parent_metadata_path, "r", encoding="utf-8") as f:
    parent_params_f02 = json.load(f)

# =====================================================================
# 5. Resolver Tu (prioridad: F03 → F02)
# =====================================================================
def resolve_Tu(params_f03, parent_params_f02):
    """
    Selección de Tu:
      1. Explícito en la variante de Fase 03
      2. Heredado de metadata de Fase 02
      3. Error si no existe
    """
    if params_f03.get("Tu") is not None:
        Tu = float(params_f03["Tu"])
        print(f"✔ Tu tomado de variante Fase 03: {Tu}")
        return Tu

    Tu_f02 = parent_params_f02.get("Tu")
    if Tu_f02 is not None:
        Tu = float(Tu_f02)
        print(f"✔ Tu heredado de metadata Fase 02: {Tu}")
        return Tu

    raise RuntimeError(
        "❌ No se pudo determinar Tu (ni en Fase 03 ni en Fase 02)."
    )


Tu = resolve_Tu(params_f03, parent_params_f02)
print(f"👉 Tu final = {Tu} segundos")


✔ Parámetros de Fase 03 validados correctamente según schema.
────────── VARIANT PARAMETERS (F03) ──────────
variant_id        = v111
parent_phase      = 02_prepareeventsds
parent_variant    = v011
OW                = 10
LT                = 1
PW                = 2
window_strategy   = synchro
nan_strategy      = discard
──────────────────────────────────────────────
✔ Tu heredado de metadata Fase 02: 10.0
👉 Tu final = 10.0 segundos


In [5]:
# =====================================================================
# 6. Localización del dataset de entrada (salida Fase 02)
# =====================================================================
from pathlib import Path

input_dataset_path = (
    ctx["project_root"]
    / "params"
    / parent_phase            # 02_prepareeventsds
    / parent_variant
    / f"{parent_phase}_dataset.parquet"
)

if not input_dataset_path.exists():
    raise FileNotFoundError(
        f"No se encontró el dataset de entrada de la Fase 02:\n"
        f"  {input_dataset_path}\n"
        f"¿Se ejecutó correctamente {parent_phase} "
        f"para la variante {parent_variant}?"
    )

print(f"✔ Dataset de entrada (F02): {input_dataset_path}")


✔ Dataset de entrada (F02): /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/02_prepareeventsds/v011/02_prepareeventsds_dataset.parquet


In [6]:
# =====================================================================
# 7. Carga eficiente del dataset base (Fase 02)
# =====================================================================
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# Lectura eficiente con PyArrow
table = pq.read_table(input_dataset_path)

# Conversión controlada a pandas
df = table.to_pandas(
    split_blocks=True,     # menos fragmentación de memoria
    self_destruct=True     # libera buffers Arrow tras la conversión
)

print(f"✔ Dataset cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")

# ---------------------------------------------------------------------
# Comprobaciones mínimas y preparación
# ---------------------------------------------------------------------

# 1) Columna temporal obligatoria
if "segs" not in df.columns:
    raise ValueError("El dataset no contiene la columna temporal obligatoria 'segs'")

# 2) Orden temporal (barato y estable)
if not df["segs"].is_monotonic_increasing:
    df = df.sort_values("segs", kind="mergesort").reset_index(drop=True)
    print("ℹ Dataset ordenado por 'segs'")

# 3) Conversión a numpy para acceso rápido
time_array = df["segs"].to_numpy(dtype=np.int64, copy=False)


✔ Dataset cargado: 3,887,242 filas × 2 columnas


In [7]:
# =====================================================================
# 8. PRECOMPUTACIÓN DE DESPLAZAMIENTOS TEMPORALES (en unidades Tu)
# =====================================================================

OW_end = OW
LT_start = OW
PW_start = OW + LT
PW_end = OW + LT + PW

WINDOW_SIZE = PW_end  # ventana total máxima (en unidades Tu)

print(f"Ventana total usada = 0 → {WINDOW_SIZE * Tu} segundos")
print(f"OW, LT, PW son MULTIPLOS de Tu = {Tu} segundos")


Ventana total usada = 0 → 130.0 segundos
OW, LT, PW son MULTIPLOS de Tu = 10.0 segundos


In [8]:
# =====================================================================
# 9. PREPARACIÓN DE ESTRUCTURAS EFICIENTES
# =====================================================================
import numpy as np
from bisect import bisect_left

# Conversión de tiempo a numpy (ya ordenado)
times = df["segs"].to_numpy(dtype=np.int64, copy=False)
N = len(times)



parent_event_catalog_path = (
    ctx["project_root"]
    / "params"
    / parent_phase
    / parent_variant
    / f"{parent_phase}_event_catalog.json"
)

if not parent_event_catalog_path.exists():
    raise FileNotFoundError(
        f"No se encontró catálogo de eventos de la fase padre:\n{parent_event_catalog_path}"
    )

with open(parent_event_catalog_path, "r", encoding="utf-8") as f:
    parent_event_catalog_f02 = json.load(f)

nan_event_codes = {
    code
    for name, code in parent_event_catalog_f02.items()
    if name.strip().endswith("_NaN_NaN")
}

nan_event_codes = set(nan_event_codes)  # ya lo es, pero explícito

has_nan = np.array(
    [
        any(ev in nan_event_codes for ev in ev_list)
        if ev_list is not None else False
        for ev_list in df["events"]
    ],
    dtype=bool,
)

print("Eventos con NaN en catálogo:")
for name, code in parent_event_catalog_f02.items():
    if "NaN" in name:
        print(name, "->", code)


# Detección rápida de NaN en eventos
#has_nan = np.isin(df["events"].to_numpy(), list(nan_event_codes))

print(f"Dataset preparado: {N:,} timestamps")


Eventos con NaN en catálogo:
Battery_Active_Power_NaN_NaN -> 73
Battery_Active_Power_Set_Response_NaN_NaN -> 146
PVPCS_Active_Power_NaN_NaN -> 219
GE_Body_Active_Power_NaN_NaN -> 292
GE_Active_Power_NaN_NaN -> 365
GE_Body_Active_Power_Set_Response_NaN_NaN -> 438
FC_Active_Power_FC_END_Set_NaN_NaN -> 511
FC_Active_Power_NaN_NaN -> 584
FC_Active_Power_FC_end_Set_Response_NaN_NaN -> 657
Island_mode_MCCB_Active_Power_NaN_NaN -> 730
MG-LV-MSB_AC_Voltage_NaN_NaN -> 803
Receiving_Point_AC_Voltage_NaN_NaN -> 876
Island_mode_MCCB_AC_Voltage_NaN_NaN -> 949
Island_mode_MCCB_Frequency_NaN_NaN -> 1022
MG-LV-MSB_Frequency_NaN_NaN -> 1095
Inlet_Temperature_of_Chilled_Water_NaN_NaN -> 1168
Outlet_Temperature_NaN_NaN -> 1241
Dataset preparado: 3,887,242 timestamps


In [9]:
# =====================================================================
# 10. ITERADOR DE INICIOS DE VENTANA (TODAS LAS ESTRATEGIAS)
# =====================================================================

def window_start_iterator():
    """
    Genera tiempos t0 (inicio de ventana) según la estrategia.
    Todos los tiempos están en segundos.
    """

    t_min = times[0]
    t_max = times[-1]
    total_window_sec = PW_end * Tu

    if window_strategy == "synchro":
        # Ventanas alineadas a Tu en toda la escala temporal
        t = t_min
        while t + total_window_sec <= t_max:
            yield t
            t += Tu

    elif window_strategy == "asynOW":
        # Cada evento inicia una OW
        for t in times:
            if t + total_window_sec <= t_max:
                yield t

    elif window_strategy == "withinPW":
        # Ventana cuyo PW termina en un evento
        for t in times:
            t0 = t - (OW + LT) * Tu
            if t0 >= t_min and t0 + total_window_sec <= t_max:
                yield t0

    elif window_strategy == "asynPW":
        # PW empieza exactamente en un evento
        for t in times:
            t0 = t - PW_start * Tu
            if t0 >= t_min and t0 + total_window_sec <= t_max:
                yield t0

    else:
        raise ValueError(
            f"Estrategia de enventanado no soportada: {window_strategy}"
        )


In [10]:
# =====================================================================
# 11. BUCLE PRINCIPAL — ULTRAOPTIMIZADO Y ESCALABLE
# =====================================================================

WINDOW_COUNT = 0
WRITTEN_COUNT = 0

def get_index_range(t0, t1):
    i0 = bisect_left(times, t0)
    i1 = bisect_left(times, t1)
    return i0, i1

print("Generando ventanas (sin acumular en memoria)...")

for t0 in window_start_iterator():

    t_OW_0 = t0
    t_OW_1 = t0 + OW * Tu

    t_PW_0 = t0 + PW_start * Tu
    t_PW_1 = t0 + PW_end * Tu

    i_ow_0, i_ow_1 = get_index_range(t_OW_0, t_OW_1)
    i_pw_0, i_pw_1 = get_index_range(t_PW_0, t_PW_1)

    WINDOW_COUNT += 1
    if i_ow_0 == i_ow_1 or i_pw_0 == i_pw_1:
        continue

    if nan_strategy == "discard":
        if has_nan[i_ow_0:i_ow_1].any():
            continue
        if has_nan[i_pw_0:i_pw_1].any():
            continue

    WRITTEN_COUNT += 1

print(f"✔ Total ventanas válidas generadas: {WRITTEN_COUNT:,}")


Generando ventanas (sin acumular en memoria)...


✔ Total ventanas válidas generadas: 3,884,085


In [11]:
# =====================================================================
# 12. PREPARACIÓN DE ESCRITURA INCREMENTAL (PARQUET)
# =====================================================================
import pyarrow as pa
import pyarrow.parquet as pq

OUTPUT_DATASET_PATH = (
    ctx["project_root"]
    / "params"
    / PHASE
    / ACTIVE_VARIANT
    / f"{PHASE}_dataset.parquet"
)

OUTPUT_DATASET_PATH.parent.mkdir(parents=True, exist_ok=True)

# Schema explícito (importante para estabilidad y velocidad)
schema = pa.schema([
    ("t0", pa.int64()),
    ("i_ow_0", pa.int32()),
    ("i_ow_1", pa.int32()),
    ("i_pw_0", pa.int32()),
    ("i_pw_1", pa.int32()),
])

writer = pq.ParquetWriter(
    OUTPUT_DATASET_PATH,
    schema=schema,
    compression="snappy"
)

print(f"✔ Writer Parquet preparado: {OUTPUT_DATASET_PATH}")


✔ Writer Parquet preparado: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_dataset.parquet


In [12]:
# =====================================================================
# 13. GENERACIÓN Y ESCRITURA INCREMENTAL DEL DATASET F03
# =====================================================================
import pyarrow as pa

BATCH_SIZE = 20_000   # ajustable según RAM / I/O

rows = []
DISK_WINDOW_COUNT = 0
DISK_WRITTEN_COUNT = 0

for t0 in window_start_iterator():

    # -------------------------------------------------
    # Definición de ventanas OW / PW (en tiempo)
    # -------------------------------------------------
    t_OW_0 = t0
    t_OW_1 = t0 + OW * Tu

    t_PW_0 = t0 + PW_start * Tu
    t_PW_1 = t0 + PW_end * Tu

    # -------------------------------------------------
    # Conversión a índices
    # -------------------------------------------------
    i_ow_0, i_ow_1 = get_index_range(t_OW_0, t_OW_1)
    i_pw_0, i_pw_1 = get_index_range(t_PW_0, t_PW_1)

    # -------------------------------------------------
    # Ventanas sin rango temporal válido
    # -------------------------------------------------
    if i_ow_0 == i_ow_1 or i_pw_0 == i_pw_1:
        continue

    # -------------------------------------------------
    # Descarte por eventos NaN explícitos
    # -------------------------------------------------
    if nan_strategy == "discard":
        if has_nan[i_ow_0:i_ow_1].any():
            continue
        if has_nan[i_pw_0:i_pw_1].any():
            continue

    # -------------------------------------------------
    # Fila válida
    # -------------------------------------------------
    rows.append({
        "t0": int(t0),
        "i_ow_0": int(i_ow_0),
        "i_ow_1": int(i_ow_1),
        "i_pw_0": int(i_pw_0),
        "i_pw_1": int(i_pw_1),
    })

    DISK_WINDOW_COUNT += 1

    # -------------------------------------------------
    # Escritura por lotes
    # -------------------------------------------------
    if len(rows) >= BATCH_SIZE:
        table = pa.Table.from_pylist(rows, schema=schema)
        writer.write_table(table)
        DISK_WRITTEN_COUNT += len(rows)
        rows.clear()

# -----------------------------------------------------
# Flush final
# -----------------------------------------------------
if rows:
    table = pa.Table.from_pylist(rows, schema=schema)
    writer.write_table(table)
    DISK_WRITTEN_COUNT += len(rows)
    rows.clear()

writer.close()

print(f"✔ Ventanas válidas generadas : {DISK_WINDOW_COUNT:,}")
print(f"✔ Filas escritas en Parquet  : {DISK_WRITTEN_COUNT:,}")


✔ Ventanas válidas generadas : 3,884,085
✔ Filas escritas en Parquet  : 3,884,085


In [13]:
# =====================================================================
# 14. SANITY CHECKS DEL DATASET F03
# =====================================================================
import pyarrow.parquet as pq
import numpy as np

# 1) Abrir parquet sin cargar todo
pq_file = pq.ParquetFile(OUTPUT_DATASET_PATH)

num_rows = pq_file.metadata.num_rows
num_row_groups = pq_file.metadata.num_row_groups
schema_read = pq_file.schema_arrow

print("✔ Dataset F03 accesible")
print(f"  Filas totales        : {num_rows:,}")
print(f"  Row groups           : {num_row_groups}")
print(f"  Schema               : {schema_read}")

# 2) Leer un batch pequeño para checks rápidos
sample = pq_file.read_row_group(0).to_pandas()

required_cols = {"t0", "i_ow_0", "i_ow_1", "i_pw_0", "i_pw_1"}
missing = required_cols - set(sample.columns)
if missing:
    raise ValueError(f"Faltan columnas obligatorias en F03: {missing}")

# 3) Checks de coherencia básica
assert (sample["i_ow_0"] < sample["i_ow_1"]).all(), "OW índices no válidos"
assert (sample["i_pw_0"] < sample["i_pw_1"]).all(), "PW índices no válidos"
assert (sample["i_ow_1"] <= sample["i_pw_0"]).all(), "OW y PW solapan incorrectamente"

# 4) Checks temporales rápidos
t0_min = sample["t0"].min()
t0_max = sample["t0"].max()
print(f"✔ Rango t0 (muestra): {t0_min} → {t0_max}")

# 5) Estimación de densidad de ventanas
if num_rows > 0:
    approx_step = np.diff(sample["t0"].values[:10]).mean() if len(sample) > 10 else None
    print(f"✔ Paso temporal medio (muestra): {approx_step}")

print("✔ Sanity checks F03 completados correctamente")


✔ Dataset F03 accesible
  Filas totales        : 3,884,085
  Row groups           : 195
  Schema               : t0: int64
i_ow_0: int32
i_ow_1: int32
i_pw_0: int32
i_pw_1: int32
✔ Rango t0 (muestra): 1651363201 → 1651563191
✔ Paso temporal medio (muestra): 10.0
✔ Sanity checks F03 completados correctamente


In [14]:
# =====================================================================
# 15. GENERACIÓN DE METADATA DE FASE 03
# =====================================================================
import json
from datetime import datetime, timezone
import pyarrow.parquet as pq

# ---------------------------------------------------------------------
# Información básica del dataset generado
# ---------------------------------------------------------------------
pq_file = pq.ParquetFile(OUTPUT_DATASET_PATH)

num_windows = pq_file.metadata.num_rows
num_row_groups = pq_file.metadata.num_row_groups

# Leer un pequeño sample para stats temporales
sample = pq_file.read_row_group(0).to_pandas()

t0_min = int(sample["t0"].min()) if num_windows > 0 else None
t0_max = int(sample["t0"].max()) if num_windows > 0 else None

# ---------------------------------------------------------------------
# Construcción del metadata
# ---------------------------------------------------------------------
metadata_f03 = {
    "phase": PHASE,
    "variant_id": variant_id,
    "parent_phase": parent_phase,
    "parent_variant": parent_variant,

    # Parámetros temporales efectivos
    "Tu_seconds": Tu,
    "OW_units": OW,
    "LT_units": LT,
    "PW_units": PW,
    "OW_seconds": OW * Tu,
    "LT_seconds": LT * Tu,
    "PW_seconds": PW * Tu,
    "total_window_seconds": (OW + LT + PW) * Tu,

    # Estrategias
    "window_strategy": window_strategy,
    "nan_strategy": nan_strategy,

    # Dataset generado
    "dataset_path": str(OUTPUT_DATASET_PATH),
    "num_windows": int(num_windows),
    "num_row_groups": int(num_row_groups),

    # Información temporal (parcial, sample)
    "t0_min_sample": t0_min,
    "t0_max_sample": t0_max,

    # Trazabilidad
    "generated_at": datetime.now(timezone.utc).isoformat(),
    "generator": "03_preparewindowsds.ipynb",

    "windows_candidates": int(WINDOW_COUNT),
    "windows_discarded_nan": int(WRITTEN_COUNT) - int(WINDOW_COUNT),
    "windows_kept": int(WINDOW_COUNT),
    "discard_nan_ratio": (
        (float(WRITTEN_COUNT) - float(WINDOW_COUNT)) / float(WRITTEN_COUNT)
        if WRITTEN_COUNT > 0 else None
    ),
    "disk_windows_count": int(DISK_WINDOW_COUNT),
    "disk_written_count": int(DISK_WRITTEN_COUNT),

}

# ---------------------------------------------------------------------
# Guardado de metadata
# ---------------------------------------------------------------------
metadata_path = (
    ctx["project_root"]
    / "params"
    / PHASE
    / ACTIVE_VARIANT
    / f"{PHASE}_metadata.json"
)

with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(metadata_f03, f, indent=2, ensure_ascii=False)

print(f"✔ Metadata F03 generada: {metadata_path}")
print(f"✔ Ventanas totales: {num_windows:,}")


✔ Metadata F03 generada: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_metadata.json
✔ Ventanas totales: 3,884,085


In [15]:
# =====================================================================
# ESTADÍSTICOS DEL DATASET F03 (STREAMING, SIN CARGAR TODO)
# =====================================================================
import json
import pyarrow.parquet as pq
import numpy as np
from datetime import datetime, timezone

# Ruta de salida DEFINITIVA
STATS_PATH = VARIANT_DIR / "03_preparewindowsds_stats.json"

pq_file = pq.ParquetFile(OUTPUT_DATASET_PATH)

n_windows = pq_file.metadata.num_rows
n_row_groups = pq_file.metadata.num_row_groups

# Estadísticos básicos
t0_min = None
t0_max = None

# Para percentiles (muestreo controlado)
SAMPLE_MAX = 300_000
sample_ow = []
sample_pw = []
sample_step = []

last_t0 = None
rng = np.random.default_rng(123)

for rg in range(n_row_groups):
    df_rg = pq_file.read_row_group(
        rg,
        columns=["t0", "i_ow_0", "i_ow_1", "i_pw_0", "i_pw_1"]
    ).to_pandas(split_blocks=True)

    if df_rg.empty:
        continue

    # Min / Max temporal
    rg_min = int(df_rg["t0"].min())
    rg_max = int(df_rg["t0"].max())
    t0_min = rg_min if t0_min is None else min(t0_min, rg_min)
    t0_max = rg_max if t0_max is None else max(t0_max, rg_max)

    # Longitudes OW / PW (en nº de timestamps)
    ow_len = (df_rg["i_ow_1"] - df_rg["i_ow_0"]).to_numpy()
    pw_len = (df_rg["i_pw_1"] - df_rg["i_pw_0"]).to_numpy()

    remaining = SAMPLE_MAX - len(sample_ow)
    if remaining > 0:
        take = min(remaining, len(df_rg))
        if len(df_rg) > take:
            idx = rng.choice(len(df_rg), size=take, replace=False)
            sample_ow.extend(ow_len[idx].tolist())
            sample_pw.extend(pw_len[idx].tolist())
        else:
            sample_ow.extend(ow_len.tolist())
            sample_pw.extend(pw_len.tolist())

    # Pasos temporales (Δt0)
    t0_arr = df_rg["t0"].to_numpy(dtype=np.int64, copy=False)
    if last_t0 is not None:
        sample_step.append(int(t0_arr[0] - last_t0))
    if len(t0_arr) > 1:
        diffs = np.diff(t0_arr)
        k = min(2000, len(diffs))
        if len(diffs) > k:
            idx2 = rng.choice(len(diffs), size=k, replace=False)
            sample_step.extend(diffs[idx2].tolist())
        else:
            sample_step.extend(diffs.tolist())
    last_t0 = int(t0_arr[-1])

def percentiles(arr, ps=(0, 5, 25, 50, 75, 95, 100)):
    if not arr:
        return None
    arr = np.asarray(arr)
    return {f"p{p}": float(np.percentile(arr, p)) for p in ps}

stats = {
    "phase": PHASE,
    "variant_id": ACTIVE_VARIANT,
    "dataset_path": str(OUTPUT_DATASET_PATH),

    "num_windows": int(n_windows),
    "row_groups": int(n_row_groups),

    "t0_min": int(t0_min) if t0_min is not None else None,
    "t0_max": int(t0_max) if t0_max is not None else None,

    "ow_len_idx_percentiles": percentiles(sample_ow),
    "pw_len_idx_percentiles": percentiles(sample_pw),
    "t0_step_seconds_percentiles": percentiles(sample_step),

    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "windows_candidates": int(WINDOW_COUNT),
    "windows_discarded_nan": int(WINDOW_COUNT) - int(WRITTEN_COUNT),
    "windows_kept": int(WINDOW_COUNT),
    "discard_nan_ratio": (
        (int(WINDOW_COUNT) - int(WRITTEN_COUNT)) / float(WINDOW_COUNT)
        if WINDOW_COUNT > 0 else None
    ),

}

with open(STATS_PATH, "w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)

print(f"✔ Estadísticas F03 generadas en: {STATS_PATH}")
print(f"✔ Ventanas totales: {n_windows:,}")


✔ Estadísticas F03 generadas en: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_stats.json
✔ Ventanas totales: 3,884,085


In [16]:
# =====================================================================
# 16. PREPARAR RUTAS DE INFORME (HTML + FIGURAS)
#     usando save_figure() de figures.py
# =====================================================================
from pathlib import Path

# Importar utilidad común de figuras
from mlops4ofp.tools.figures import save_figure

# Directorios de reporte
REPORT_DIR = VARIANT_DIR

FIG_DIR = VARIANT_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"✔ Report dir: {REPORT_DIR}")
print(f"✔ Fig dir   : {FIG_DIR}")


✔ Report dir: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111
✔ Fig dir   : /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/figures


In [17]:
# =====================================================================
# 17. PREPARACIÓN DE MUESTRAS PARA GRÁFICAS (STREAMING)
# =====================================================================
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
from collections import Counter

pq_file = pq.ParquetFile(OUTPUT_DATASET_PATH)

cols = ["t0", "i_ow_0", "i_ow_1", "i_pw_0", "i_pw_1"]

# ---------------------------------------------------------------------
# Contenedores de salida (los usará la celda de gráficas)
# ---------------------------------------------------------------------
SAMPLE_MAX = 300_000

sample_ow = []
sample_pw = []
sample_step = []

windows_per_hour = Counter()

last_t0 = None
rng = np.random.default_rng(123)

# ---------------------------------------------------------------------
# Lectura streaming por row-groups
# ---------------------------------------------------------------------
for rg in range(pq_file.metadata.num_row_groups):
    df_rg = pq_file.read_row_group(rg, columns=cols).to_pandas(split_blocks=True)

    if df_rg.empty:
        continue

    # Longitudes OW / PW (en nº de timestamps)
    ow_len = (df_rg["i_ow_1"].to_numpy() - df_rg["i_ow_0"].to_numpy())
    pw_len = (df_rg["i_pw_1"].to_numpy() - df_rg["i_pw_0"].to_numpy())

    # Muestreo controlado
    remaining = SAMPLE_MAX - len(sample_ow)
    if remaining > 0:
        take = min(remaining, len(df_rg))
        if len(df_rg) > take:
            idx = rng.choice(len(df_rg), size=take, replace=False)
            sample_ow.extend(ow_len[idx].astype(np.int32, copy=False).tolist())
            sample_pw.extend(pw_len[idx].astype(np.int32, copy=False).tolist())
        else:
            sample_ow.extend(ow_len.astype(np.int32, copy=False).tolist())
            sample_pw.extend(pw_len.astype(np.int32, copy=False).tolist())

    # Pasos temporales (Δt0)
    t0_arr = df_rg["t0"].to_numpy(dtype=np.int64, copy=False)
    if last_t0 is not None:
        sample_step.append(int(t0_arr[0] - last_t0))
    if len(t0_arr) > 1:
        diffs = np.diff(t0_arr)
        # Muestreamos diffs para no explotar memoria
        k = min(2000, len(diffs))
        if len(diffs) > k:
            idx2 = rng.choice(len(diffs), size=k, replace=False)
            sample_step.extend(diffs[idx2].astype(np.int64, copy=False).tolist())
        else:
            sample_step.extend(diffs.astype(np.int64, copy=False).tolist())
    last_t0 = int(t0_arr[-1])

    # Ventanas por hora (agregado)
    hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
    counts = hours.value_counts()
    for k, v in counts.items():
        windows_per_hour[str(k)] += int(v)

# Convertimos a numpy para las gráficas
sample_ow = np.asarray(sample_ow, dtype=np.int32)
sample_pw = np.asarray(sample_pw, dtype=np.int32)
sample_step = np.asarray(sample_step, dtype=np.int64)

print("✔ Muestras preparadas para gráficas:")
print(f"  sample_ow size   : {len(sample_ow):,}")
print(f"  sample_pw size   : {len(sample_pw):,}")
print(f"  sample_step size : {len(sample_step):,}")
print(f"  windows_per_hour : {len(windows_per_hour)} buckets")


/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt

/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt

✔ Muestras preparadas para gráficas:
  sample_ow size   : 300,000
  sample_pw size   : 300,000
  sample_step size : 390,194
  windows_per_hour : 10803 buckets


/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt.floor("H")
/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/1429257085.py:68: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = pd.to_datetime(df_rg["t0"], unit="s", utc=True).dt

In [18]:
# =====================================================================
# 17. GRÁFICAS DEL DATASET F03
#     (todas usando save_figure de figures.py)
# =====================================================================
import pandas as pd
import numpy as np

# ---------------------------------------------------------------------
# 1) Histograma de longitudes OW (span de índices)
# ---------------------------------------------------------------------
def plot_hist_ow_len():
    import matplotlib.pyplot as plt
    plt.hist(sample_ow, bins=60)
    plt.title("OW length (index span) distribution")
    plt.xlabel("i_ow_1 - i_ow_0 (num timestamps)")
    plt.ylabel("count (sample)")

save_figure(
    FIG_DIR / "hist_ow_len_idx.png",
    plot_hist_ow_len
)

# ---------------------------------------------------------------------
# 2) Histograma de longitudes PW (span de índices)
# ---------------------------------------------------------------------
def plot_hist_pw_len():
    import matplotlib.pyplot as plt
    plt.hist(sample_pw, bins=60)
    plt.title("PW length (index span) distribution")
    plt.xlabel("i_pw_1 - i_pw_0 (num timestamps)")
    plt.ylabel("count (sample)")

save_figure(
    FIG_DIR / "hist_pw_len_idx.png",
    plot_hist_pw_len
)

# ---------------------------------------------------------------------
# 3) Histograma del paso temporal entre ventanas (Δt0)
# ---------------------------------------------------------------------
def plot_hist_t0_step():
    import matplotlib.pyplot as plt
    plt.hist(sample_step, bins=60)
    plt.title("t0 step distribution (seconds)")
    plt.xlabel("Δt0 (seconds)")
    plt.ylabel("count (sample)")

save_figure(
    FIG_DIR / "hist_t0_step_seconds.png",
    plot_hist_t0_step
)

# ---------------------------------------------------------------------
# 4) Ventanas por hora (agregado)
# ---------------------------------------------------------------------
if windows_per_hour:

    def plot_windows_per_hour():
        import matplotlib.pyplot as plt

        series = pd.Series(windows_per_hour).sort_index()
        x = pd.to_datetime(series.index, utc=True)
        y = series.values

        plt.plot(x, y)
        plt.title("Windows per hour (UTC)")
        plt.xlabel("hour")
        plt.ylabel("windows")

    save_figure(
        FIG_DIR / "windows_per_hour.png",
        plot_windows_per_hour,
        figsize=(10, 4)
    )

print("✔ Figuras F03 generadas correctamente en:", FIG_DIR)


✔ Figuras F03 generadas correctamente en: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/figures


In [19]:
# =====================================================================
# 19. INFORME HTML F03 (ENLAZA FIGURAS Y ESTADÍSTICAS)
# =====================================================================
import json
from html import escape
from datetime import datetime

# ---------------------------------------------------------------------
# Rutas
# ---------------------------------------------------------------------
STATS_PATH = VARIANT_DIR / "03_preparewindowsds_stats.json"
HTML_PATH = VARIANT_DIR / "03_preparewindowsds_report.html"
print("Buscando stats en:", STATS_PATH)
print("Existe:", STATS_PATH.exists())



if not STATS_PATH.exists():
    raise FileNotFoundError(f"No se encontró el fichero de estadísticas: {STATS_PATH}")

with open(STATS_PATH, "r", encoding="utf-8") as f:
    stats = json.load(f)

# ---------------------------------------------------------------------
# Figuras a enlazar (orden y títulos)
# ---------------------------------------------------------------------
figs = [
    ("OW length distribution (index span)", "hist_ow_len_idx.png"),
    ("PW length distribution (index span)", "hist_pw_len_idx.png"),
    ("t0 step distribution (seconds)", "hist_t0_step_seconds.png"),
    ("Windows per hour (UTC)", "windows_per_hour.png"),
]

# ---------------------------------------------------------------------
# Utilidades HTML
# ---------------------------------------------------------------------
def html_table_from_dict(d):
    if not d:
        return "<em>n/a</em>"
    rows = "".join(
        f"<tr><td>{escape(str(k))}</td><td>{escape(str(v))}</td></tr>"
        for k, v in d.items()
    )
    return f"<table><tbody>{rows}</tbody></table>"

# ---------------------------------------------------------------------
# Construcción del HTML
# ---------------------------------------------------------------------
html = f"""<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8"/>
  <title>F03 Report — {escape(ACTIVE_VARIANT)}</title>
  <style>
    body {{
      font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial;
      margin: 24px;
      max-width: 1100px;
    }}
    h1, h2, h3 {{ margin-bottom: 0.3rem; }}
    .meta {{ color: #555; margin-top: 0; }}
    .card {{
      border: 1px solid #ddd;
      border-radius: 10px;
      padding: 16px;
      margin: 16px 0;
    }}
    img {{
      max-width: 100%;
      height: auto;
      border: 1px solid #eee;
      border-radius: 10px;
    }}
    table {{ border-collapse: collapse; }}
    td {{
      border: 1px solid #ddd;
      padding: 6px 10px;
    }}
    ul {{ padding-left: 18px; }}
    a {{ text-decoration: none; }}
    .small {{ color: #777; font-size: 0.9em; }}
  </style>
</head>
<body>

  <h1>F03 — PrepareWindowsDS</h1>
  <p class="meta">
    Variante: <b>{escape(ACTIVE_VARIANT)}</b><br/>
    Phase: <b>{escape(PHASE)}</b><br/>
    Dataset: <code>{escape(str(OUTPUT_DATASET_PATH))}</code><br/>
    Generado: {escape(stats.get("generated_at_utc", str(datetime.utcnow())))}
  </p>

  <div class="card">
    <h2>Resumen</h2>
    <ul>
      <li>Ventanas totales: <b>{stats.get("num_windows", 0):,}</b></li>
      <li>Row groups: <b>{stats.get("row_groups", 0)}</b></li>
      <li>t0_min: <code>{escape(str(stats.get("t0_min")))}</code></li>
      <li>t0_max: <code>{escape(str(stats.get("t0_max")))}</code></li>
      <li>Ventanas candidatas: <b>{stats.get("windows_candidates", 0):,}</b></li>
      <li>Descartadas por NaN: <b>{stats.get("windows_discarded_nan", 0):,}</b></li>
      <li>Ratio discard NaN: <b>{stats.get("discard_nan_ratio", 0):.3f}</b></li>
    </ul>
  </div>

  <div class="card">
    <h2>Percentiles</h2>

    <h3>OW length (index span)</h3>
    {html_table_from_dict(stats.get("ow_len_idx_percentiles"))}

    <h3>PW length (index span)</h3>
    {html_table_from_dict(stats.get("pw_len_idx_percentiles"))}

    <h3>Δt0 (seconds)</h3>
    {html_table_from_dict(stats.get("t0_step_seconds_percentiles"))}
  </div>

  <div class="card">
    <h2>Figuras</h2>
    <ul>
      {''.join(
        f'<li><a href="figures/{escape(fn)}">{escape(title)}</a></li>'
        for title, fn in figs
      )}
    </ul>

    {''.join(
      f'''
      <h3>{escape(title)}</h3>
      <a href="figures/{escape(fn)}">
        <img src="figures/{escape(fn)}" alt="{escape(title)}"/>
      </a>
      '''
      for title, fn in figs
    )}
  </div>

  <div class="card">
    <h2>Artefactos</h2>
    <ul>
      <li><a href="{escape(STATS_PATH.name)}">f03_stats.json</a></li>
      <li class="small">Parquet: {escape(str(OUTPUT_DATASET_PATH.name))}</li>
    </ul>
  </div>

</body>
</html>
"""

# ---------------------------------------------------------------------
# Escritura del fichero
# ---------------------------------------------------------------------
with open(HTML_PATH, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✔ Informe HTML generado: {HTML_PATH}")


Buscando stats en: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_stats.json
Existe: True
✔ Informe HTML generado: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/params/03_preparewindowsds/v111/03_preparewindowsds_report.html


/var/folders/3s/9jzs3kgs6lb46jtbw6tg9v_80000gn/T/ipykernel_63154/543699351.py:91: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  Generado: {escape(stats.get("generated_at_utc", str(datetime.utcnow())))}


## ✔ Fase 03 completada
- Dataset parquet generado
- Metadata JSON generada
- Informe HTML generado
- Listo para la siguiente fase del pipeline